## Bank Marketing Data Set

he data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls.

##### bank client data:
   1 - age (numeric)
   2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                       "blue-collar","self-employed","retired","technician","services") 
   3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
   4 - education (categorical: "unknown","secondary","primary","tertiary")
   5 - default: has credit in default? (binary: "yes","no")
   6 - balance: average yearly balance, in euros (numeric) 
   7 - housing: has housing loan? (binary: "yes","no")
   8 - loan: has personal loan? (binary: "yes","no")
##### related with the last contact of the current campaign:
   9 - contact: contact communication type (categorical: "unknown","telephone","cellular") 
  10 - day: last contact day of the month (numeric)
  11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  12 - duration: last contact duration, in seconds (numeric)
##### other attributes:
  13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
  15 - previous: number of contacts performed before this campaign and for this client (numeric)
  16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")
  
https://archive.ics.uci.edu/ml/datasets/Bank+Marketing#

#### CONEXÃO COM AWS S3 

In [73]:
import boto3  
    
import pandas as pd
from botocore.exceptions import ClientError
import logging

s3_client = boto3.client(
    's3',
    aws_access_key_id='#',
    aws_secret_access_key='#'
)


import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

#### CRIAÇÃO DO BUCKET

In [3]:

def create_bucket(bucket_name):
    try:
      s3_client.create_bucket(Bucket=bucket_name)
    except ClientError as e:
        logging.error(e)
        return False
        
    return True

create_bucket('bank-marketing')

True

#### UPLOAD DO ARQUIVO

In [7]:
def upload_object(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = file_name

    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
      
    return True

upload_object('bank/bank-full.csv', 'bank-marketing')

True

#### BUCKETS EXISTENTES

In [74]:
response = s3_client.list_buckets()

print('Buckets existentes:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Buckets existentes:
  bank-marketing


#### LISTAS OBJETOS DE UM BUCKET

In [18]:
def list_objects_bucket(bucket_name):
    response = s3_client.list_objects(Bucket=bucket_name)
    for object in response['Contents']:
        print(object['Key'])
        
list_objects_bucket('bank-marketing')

bank/bank-full.csv


#### DOWNLOADS DE OBJETOS

In [20]:
def download_object(bucket_name, object_name, file_name)
  s3_client.download_file(bucket_name, object_name, file_name)

s3_client.download_file('bank-marketing', 'bank/bank-full.csv', 'bank-full_new.csv')

In [59]:

csv_obj = s3_client.get_object(Bucket='bank-marketing', Key='bank/bank-full.csv')
body = csv_obj['Body']

In [60]:
csv_string = body.read().decode('utf-8')



In [68]:
df = pd.read_csv(StringIO(csv_string), sep=";")

In [69]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no
